In [1]:
!pip install catboost
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
import gc
from sklearn.model_selection import train_test_split

In [2]:
# 이상치 처리 X
train_df=pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/mid_train_df(2).csv')
#test_df=pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/mid_test_df(2).csv')

#sub = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/제주/submission.csv')

In [3]:
train_df.fillna(9999,inplace=True)
#test_df.fillna(9999,inplace=True)

In [4]:
train_df.drop(['CSTMR_CNT','CNT'],axis=1,inplace=True)
train_df.head()

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,AMT,STD_total,SIDO_STD_total,SIDO_CCG_STD_total,ASF_total,SIDO_ASF_total,SIDO_CCG_ASF_total,SIDO_ASF_2_total,SIDO_CCG_ASF_2_total,STD_2019,SIDO_STD_2019,SIDO_CCG_STD_2019,ASF_2019,SIDO_ASF_2019,SIDO_CCG_ASF_2019,SIDO_ASF_2_2019,SIDO_CCG_ASF_2_2019,STD_2020,SIDO_STD_2020,SIDO_CCG_STD_2020,ASF_2020,SIDO_ASF_2020,SIDO_CCG_ASF_2020,SIDO_ASF_2_2020,SIDO_CCG_ASF_2_2020
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,311200,16,258,2121,11,14,357,201,1734,10,33,2140,11,196,338.0,225.0,2027.0,13,244.0,3128.0,6,49.0,36.0,117.0,1528.0
1,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,1374500,16,258,2121,0,24,0,50,173,10,33,2140,2,53,15.0,4.0,812.0,13,244.0,3128.0,3,42.0,19.0,79.0,780.0
2,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,818700,16,258,2121,2,32,10,179,173,10,33,2140,0,5,15.0,4.0,812.0,13,244.0,3128.0,0,219.0,43.0,127.0,104.0
3,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,1717000,16,258,2121,0,24,0,41,282,10,33,2140,2,1,15.0,99.0,58.0,13,244.0,3128.0,3,42.0,742.0,114.0,3174.0
4,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,1047300,16,258,2121,0,51,0,226,14,10,33,2140,2,10,314.0,15.0,5.0,13,244.0,3128.0,3,51.0,19.0,1.0,209.0


In [5]:
# test_df 정렬
#test_df = test_df[list(train_df.drop('AMT',axis=1).columns)]
#test_df.head()

In [6]:
def make_encoders(df):
  encoders={}
  for col in df.drop(['AMT'],axis=1).columns:
    con_list =[]
    encoder={}
    for idx, content in enumerate(df[col].unique()):
      encoder[content]=idx

    if col == 'REG_YYMM':
        encoder[202005] = idx+1
        encoder[202006] = idx+2
        encoder[202007] = idx+3
     
    encoders[col] = encoder
  
  return encoders

In [7]:
# 메모리 방지를 위한 컷
train_df = train_df[train_df['REG_YYMM']>201907]

In [8]:
# public은 4월 밖에 계산을 못함
df_not04=train_df[train_df['REG_YYMM'] != 202004]
df_202004=train_df[train_df['REG_YYMM'] == 202004]

In [9]:
# 인코딩
encoders = make_encoders(train_df)

In [10]:
for col in encoders.keys():
  df_not04[col] = df_not04[col].map(encoders[col])
  df_202004[col] = df_202004[col].map(encoders[col])
  #train_df[col] = train_df[col].map(encoders[col])
  #df_202005[col] = df_202005[col].map(encoders[col])
  #df_202006[col] = df_202006[col].map(encoders[col])
  #df_202007[col] = df_202007[col].map(encoders[col])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
for col in train_df.columns:
  if col !='AMT':
    df_not04[col] = df_not04[col].astype('object')
    df_202004[col] = df_202004[col].astype('object')
    #train_df[col] = train_df[col].astype('object')
    #df_202005[col] = df_202005[col].astype('object')
    #df_202006[col] = df_202006[col].astype('object')
    #df_202007[col] = df_202007[col].astype('object')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [12]:
# X_train, X_val, y_train, y_val = train_test_split(df_not04.drop('AMT',axis=1),
#                                                   np.log1p(df_not04['AMT']), test_size=0.3, random_state=42)

In [13]:
del train_df
gc.collect()

167

catboost 파라미터: bagging_temperature ,
              depth , 
                  learning_rate ,
                  min_data_in_leaf , 
                  max_leaves , 
                  l2_leaf_reg , 
                  border_count

In [14]:
cat_features = list(df_not04.drop('AMT',axis=1).columns)

model = CatBoostRegressor(loss_function="RMSE",
                           eval_metric="RMSE",
                           task_type="GPU",
                           learning_rate=0.01,
                           iterations=150,
                           boosting_type='Plain', # 메모리 과부하 방지
                           #max_ctr_complexity=2,
                           gpu_cat_features_storage = 'CpuPinnedMemory',
                           early_stopping_rounds=10)



model.fit(df_not04.drop('AMT',axis=1), np.log1p(df_not04['AMT']),
          cat_features=cat_features,
          eval_set=(df_202004.drop('AMT',axis=1),np.log1p(df_202004['AMT'])),
          use_best_model=True,
          verbose = 10)

0:	learn: 1.7119729	test: 1.7184083	best: 1.7184083 (0)	total: 120ms	remaining: 17.9s
10:	learn: 1.6937132	test: 1.7016691	best: 1.7016691 (10)	total: 1.09s	remaining: 13.8s
20:	learn: 1.6780857	test: 1.6873354	best: 1.6873354 (20)	total: 2.07s	remaining: 12.7s
30:	learn: 1.6615658	test: 1.6731294	best: 1.6731294 (30)	total: 3.01s	remaining: 11.6s
40:	learn: 1.6453949	test: 1.6584440	best: 1.6584440 (40)	total: 3.95s	remaining: 10.5s
50:	learn: 1.6315535	test: 1.6459561	best: 1.6459561 (50)	total: 4.9s	remaining: 9.52s
60:	learn: 1.6197798	test: 1.6353864	best: 1.6353864 (60)	total: 5.84s	remaining: 8.52s
70:	learn: 1.6095644	test: 1.6263427	best: 1.6263427 (70)	total: 6.8s	remaining: 7.56s
80:	learn: 1.5991161	test: 1.6169170	best: 1.6169170 (80)	total: 7.77s	remaining: 6.62s
90:	learn: 1.5887245	test: 1.6074283	best: 1.6074283 (90)	total: 8.76s	remaining: 5.68s
100:	learn: 1.5808515	test: 1.6004101	best: 1.6004101 (100)	total: 9.72s	remaining: 4.72s
110:	learn: 1.5733570	test: 1.5936

In [15]:
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,STD_CLSS_NM,24.731392
1,SIDO_CCG_STD_2020,21.859502
2,HOM_SIDO_NM,7.607103
3,CARD_SIDO_NM,7.488098
4,HOM_CCG_NM,6.586115
5,CARD_CCG_NM,4.968020
6,SIDO_CCG_ASF_2_2020,4.264329
7,SIDO_CCG_ASF_2_total,3.556321
8,SIDO_CCG_STD_total,3.262123
9,SIDO_CCG_STD_2019,2.957866
